### Steps for RAG
- give URL of where Law is being contradicted and give citation 

- add parameter in json for: citation/URL of law and explanation of law
- rag/web-scraping of URL and grab the info about the Law to ensure that the law exists
	- if nothing is retrieved back then law does not exists and skip file(re-run later)
- pass the rag output(contains the web-scraped information about the law, definition) and pass the explanation object of the json-file into LLM
	- LLM prompt: compares both explanation to ensure that the contradicted law validates the reason for the perturbation. 
		- ask LLM to provide metric output(confidence score)
		- set threshold 

### Ambiguity Legal

In [ ]:
f"""You are a senior compliance officer reviewing a legal contract. Your task is to modify a section of the contract by introducing an ambiguous legal obligation while ensuring that the ambiguity contradicts a state or national law.
    
    Before modifying the text:
    - **Read the file** to determine what city, state, or country the contract applies to.
    - If the jurisdiction is unclear, default to **United States law**.
    - Make sure that when taking the original texts, there should be no jumps between sentences. Take the start to end of the original section without skipping sentences.
    
    ### **Definition:**
    Ambiguities occur when a legal statement is vague, leading to multiple interpretations. A **legal contradiction** under this category happens when an obligation is introduced ambiguously, making it difficult to enforce under state or national law. This can result in non-compliance with regulatory requirements, leaving legal obligations open to dispute.
    
    ### **Step-by-Step Instructions:**
    1. Identify a clear legal obligation in the contract.
    2. Modify the wording to make it **vague or open to multiple interpretations**.
    3. Ensure that this ambiguity creates **non-compliance with a specific law** in the identified jurisdiction.
    4. Use online legal databases or verified government resources to **find a real law that is contradicted**.
    5. For that contradiction, extract:
   - **The citation/title of the law** (e.g., "29 CFR § 516.2" or "CA Labor Code § 201")
   - **Two direct links (URLs)** to official or government sources for the law (**these links must be accessible. The relevant citation should be inside this URL. The page should be found - we should not get a Page Not Found or similar error.**)
   - **The first URL must come from an official government or legislature domain (ends in .gov, .mil, .state.<XX>.us, or one of uscode.house.gov, ecfr.gov, govinfo.gov, law.cornell.edu). Use the most recent version of the statute/regulation.**
   - **The second URL should be a secondary link to serve as a backup in case the first link falters, but we wish for that to not happen.**
   - **A brief explanation** of how the modified text contradicts the law (**law_explanation**)
    6. Make 2–3 such perturbations for the file.
    7. Output only the structured JSON object as shown below.
    
    ---
    
    ### **Examples of Ambiguous Legal Obligations:**
    
    **Example 1:**
    - **Original:** "The company shall provide necessary accommodations for disabled employees."
    - **Modified:** "The company shall provide accommodations for disabled employees as deemed appropriate."
    - **Explanation:** The term "as deemed appropriate" introduces ambiguity, conflicting with **ADA (Americans with Disabilities Act)**, which mandates **clear, non-discretionary accommodations**.
    
    **Example 2:**
    - **Original:** "All contractors must comply with local zoning laws."
    - **Modified:** "All contractors must make reasonable efforts to comply with zoning laws."
    - **Explanation:** "Reasonable efforts" is vague—some zoning laws require strict adherence.
    
    **Example 3:**
    - **Original:** "The landlord shall ensure habitable living conditions in compliance with state law."
    - **Modified:** "The landlord shall make efforts to maintain habitable conditions."
    - **Explanation:** "Make efforts" does not guarantee habitability, which violates **tenant protection laws**.
    
    **Example 4:**
    - **Original:** "The company shall maintain data security measures that meet industry standards."
    - **Modified:** "The company shall maintain data security measures that it deems sufficient."
    - **Explanation:** "Deems sufficient" is subjective and contradicts **GDPR and CCPA** requirements for **specific security standards**.
    
    **Example 5:**
    - **Original:** "Employees shall be provided meal breaks as required by law."
    - **Modified:** "Employees shall be encouraged to take meal breaks."
    - **Explanation:** Some states require **mandatory** meal breaks (e.g., California).
    
    **Example 6:**
    - **Original:** "The Contractor shall comply with all federal and state regulations governing workplace safety, ensuring all necessary precautions are taken to protect employees from occupational hazards. The Contractor must conduct quarterly safety inspections and submit reports to regulatory authorities. Any violations of safety standards shall result in corrective actions and potential penalties. The company’s leadership is responsible for ensuring full compliance at all levels."
    - **Modified:** "The Contractor should make reasonable efforts to comply with applicable federal and state regulations governing workplace safety. The Contractor may conduct periodic safety inspections and submit reports when deemed necessary. Violations of safety standards will be assessed on a case-by-case basis, and corrective actions may be recommended where appropriate."
    - **Explanation:** This change weakens the legal obligation by replacing 'shall comply' with 'should make reasonable efforts,' making compliance discretionary rather than mandatory. The removal of 'quarterly safety inspections' eliminates a clear legal requirement, and replacing 'shall result in corrective actions' with 'may be recommended' creates uncertainty. This contradicts **OSHA regulations**, which mandate strict compliance and routine reporting on workplace safety violations.
    ---
    
    
    ### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Ambiguities - Ambiguous Legal Obligation",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "contradicted_law": "SPECIFIC LAW OR REGULATION BEING VIOLATED",
                "law_citation": "TITLE OR SECTION OF THE LAW (e.g., '29 CFR § 516.2')",
                "law_url1": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_1"
                ],
                "law_url2": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_2"
                ],
                "law_explanation": "HOW AND WHY THIS MODIFICATION CONTRADICTS THAT LAW",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}
    
    
    Below is the original legal text:
    -------------------
    {original_text}
    -------------------
    
    Now, return ONLY the structured JSON object with the modified text and explanation.
    """


### Misaligned Terminology

In [ ]:
f"""You are a state regulatory compliance officer ensuring that legal contracts align with statutory definitions. Your task is to modify a contract so that a key term contradicts its definition under state or federal law.
    
    Before modifying the text:
    - **Read the file** to determine what city, state, or country the contract applies to.
    - If the jurisdiction is unclear, default to **United States law**.
    - Make sure that when taking the original texts, there should be no jumps between sentences. Take the start to end of the original section without skipping sentences.
    
    ### **Definition:**
    Misaligned terminology occurs when a contract **uses terms that do not match their statutory meaning**, leading to non-compliance. A **legal contradiction** in this category arises when contractual language deviates from legally recognized definitions, making the contract unenforceable.
    
    ### **Step-by-Step Instructions:**
    1. Identify a **key term** in the contract.
    2. Modify its definition in different sections so that they **conflict**.
    3. Ensure the contradiction creates **uncertainty in enforcement**.
    4. Use online legal databases or verified government resources to **find a real law that is contradicted**.
    5. For that contradiction, extract:
    - **The citation/title of the law** (e.g., "29 CFR § 516.2" or "CA Labor Code § 201")
    - **Two direct links (URLs)** to official or government sources for the law (**these links must be accessible. The relevant citation should be inside this URL. The page should be found - we should not get a Page Not Found or similar error.**)
    - **The first URL must come from an official government or legislature domain (ends in .gov, .mil, .state.<XX>.us, or one of uscode.house.gov, ecfr.gov, govinfo.gov, law.cornell.edu). Use the most recent version of the statute/regulation.**
    - **The second URL should be a secondary link to serve as a backup in case the first link falters, but we wish for that to not happen.**
    - **A brief explanation** of how the modified text contradicts the law (**law_explanation**)
    6. Make 2–3 such perturbations for the file.
    7. Output only the structured JSON object as shown below.
    
    ---
    
    ### **Examples of Conflicting Contract and Statutory Terms:**
    
    **Example 1:**
    - **Original:** "The employee shall be considered an independent contractor."
    - **Modified:** "The worker shall be considered an independent contractor."
    - **Explanation:** Some states define **“worker” differently from “independent contractor”** under employment laws, leading to misclassification.
    
    **Example 2:**
    - **Original:** "Bank deposits are federally insured up to $250,000."
    - **Modified:** "Bank deposits may be insured at the institution’s discretion."
    - **Explanation:** This contradicts **FDIC insurance laws**, which guarantee deposit protection.
    
    **Example 3:**
    - **Original:** "A tenant has the right to a habitable living environment."
    - **Modified:** "A tenant has the right to request habitability improvements."
    - **Explanation:** Federal **tenant protection laws** mandate a **clear landlord obligation**, which this change removes.
    
    **Example 4:**
    - **Original:** "Employees are entitled to paid overtime after 40 hours per week."
    - **Modified:** "Overtime pay shall be determined based on business needs."
    - **Explanation:** The **Fair Labor Standards Act (FLSA)** mandates **fixed overtime rules**—this modification makes it discretionary.
    
    **Example 5:**
    - **Original:** "All sales agreements must disclose total finance charges."
    - **Modified:** "Finance charge disclosure is encouraged but not required."
    - **Explanation:** The **Truth in Lending Act (TILA)** requires **clear disclosure** of finance charges.
    
    **Example 6:**
    - **Original:** "The lessee shall be responsible for all maintenance and repair costs associated with the leased property, except those classified as structural repairs, which shall be the responsibility of the lessor. Structural repairs include foundation, roof, and exterior load-bearing walls."
    - **Modified:** "The lessee shall be responsible for all upkeep and general property repairs, except for major repairs, which shall be handled by the lessor."
    - **Explanation:** The term **'structural repairs'** was replaced with **'major repairs'**, creating legal uncertainty. Structural repairs have a clear **legal definition**, whereas 'major repairs' is vague and open to interpretation. This contradicts the intent of the contract and could lead to disputes over **repair responsibilities** under real estate law.
    
    ---
    
    ### **Return JSON Format**
        ### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Misaligned Terminology - Legal Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "contradicted_law": "SPECIFIC LAW OR REGULATION BEING VIOLATED",
                "law_citation": "TITLE OR SECTION OF THE LAW (e.g., '29 CFR § 516.2')",
                "law_url1": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_1"
                ],
                "law_url2": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_2"
                ],
                "law_explanation": "HOW AND WHY THIS MODIFICATION CONTRADICTS THAT LAW",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}
    
    
    Below is the original legal text:
    -------------------
    {original_text}
    -------------------
    
    Now, return ONLY the structured JSON object with the modified text and explanation.
    """

## Omission Legal

In [ ]:
f"""
You are a corporate legal advisor reviewing a contract for compliance. Your task is to introduce a contractual omission that removes a legally required consumer right, thereby violating state or national law.

Before modifying the text:
- **Read the file** to determine what city, state, or country the contract applies to.
- If the jurisdiction is unclear, default to **United States law**.
- Make sure that when taking the original texts, there should be no jumps between sentences. Take the start to end of the original section without skipping sentences.

### **Definition:**
Omissions occur when a contract **removes essential information**, creating legal loopholes. A **legal contradiction** in this category happens when a contract omits **a legally mandated consumer protection**, making it non-compliant.

### **Step-by-Step Instructions:**
    1. Identify a clause that **grants consumers a legally required right**.
    2. **Remove or modify** it so that the contract becomes **non-compliant with a law**.
    3. Ensure that the omission introduces **legal risk** in the identified jurisdiction.
    4. Use **online legal databases or verified government resources** to **find a real law** that is contradicted by this omission.
    5. For that contradiction, extract:
    - **The citation/title of the law** (e.g., "29 CFR § 516.2" or "CA Labor Code § 201")
    - **Two direct links (URLs)** to official or government sources for the law (**these links must be accessible. The relevant citation should be inside this URL. The page should be found - we should not get a Page Not Found or similar error.**)
    - **The first URL must come from an official government or legislature domain (ends in .gov, .mil, .state.<XX>.us, or one of uscode.house.gov, ecfr.gov, govinfo.gov, law.cornell.edu). Use the most recent version of the statute/regulation.**
    - **The second URL should be a secondary link to serve as a backup in case the first link falters, but we wish for that to not happen.**
    - **A brief explanation** of how the modified text contradicts the law (**law_explanation**)
    6. Make 2–3 such perturbations for the file.
    7. Output only the structured JSON object as shown below.
    
    ---
### **Examples of Omitting Consumer Rights:**

**Example 1:**
- **Original:** "Customers have the right to return the product within 30 days, as required by state law."
- **Modified:** "Customers have the right to return the product at the company’s discretion."
- **Explanation:** Many states (e.g., California) require **minimum return periods**—removing this makes the contract non-compliant.

**Example 2:**
- **Original:** "The landlord shall provide a habitable living environment as required by law."
- **Modified:** "The landlord shall make reasonable efforts to provide a habitable living environment."
- **Explanation:** "Reasonable efforts" **weakens tenant rights**, violating **housing laws**.

**Example 3:**
- **Original:** "Bank deposits are insured up to $250,000 under federal law."
- **Modified:** "Bank deposits may be insured based on company policy."
- **Explanation:** Federal law **mandates insurance coverage**—removing this statement misleads consumers.

**Example 4:**
- **Original:** "Employees are entitled to paid overtime after 40 hours per week."
- **Modified:** "Overtime compensation shall be determined at the company's discretion."
- **Explanation:** **Federal labor laws** require **clear overtime pay guidelines**.

**Example 5:**
- **Original:** "All loan agreements must provide an Annual Percentage Rate (APR) disclosure."
- **Modified:** "Loan agreements may include an APR disclosure."
- **Explanation:** Some laws **mandate APR transparency**—omitting it makes loans non-compliant.

**Example 6:**
- **Original:** "The tenant shall maintain comprehensive liability insurance covering all damages arising from negligence or misconduct on the premises, with a minimum coverage amount of $2 million per occurrence. The tenant shall provide proof of insurance annually to the landlord. Failure to maintain coverage at this level shall constitute grounds for lease termination."
- **Modified:** "The tenant shall maintain liability insurance covering damages arising from negligence or misconduct on the premises. Proof of insurance may be requested by the landlord as necessary."
- **Explanation:** The removal of 'comprehensive' and 'minimum coverage amount' significantly weakens the insurance requirement, and deleting the **annual proof requirement** removes an important compliance measure. This creates a **legal contradiction** with state laws mandating specific minimum coverage amounts for commercial leases.

---
### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Omissions - Omission Legal Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "contradicted_law": "SPECIFIC LAW OR REGULATION BEING VIOLATED",
                "law_citation": "TITLE OR SECTION OF THE LAW (e.g., '29 CFR § 516.2')",
                "law_url1": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_1"
                ],
                "law_url2": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_2"
                ],
                "law_explanation": "HOW AND WHY THIS MODIFICATION CONTRADICTS THAT LAW",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}

Below is the original legal text:
-------------------
{original_text}
-------------------

Now, return **ONLY** the structured JSON object with the modified text and explanation.
"""


## Structural Legal

In [ ]:
f"""
You are a regulatory compliance officer ensuring that legal disclosures are properly placed within contracts. Your task is to relocate a legally required disclosure to an irrelevant section, making it difficult to enforce.

Before modifying the text:
- **Read the file** to determine what city, state, or country the contract applies to.
- If the jurisdiction is unclear, default to **United States law**.
- Make sure that when taking the original texts, there should be no jumps between sentences. Take the start to end of the original section without skipping sentences.

### **Definition:**
Structural flaws occur when the **organization of a contract affects its clarity or enforceability**. A **legal contradiction** in this category arises when a required legal disclosure is **placed in an irrelevant or misleading section**, making it difficult for parties to locate or interpret.

### **Step-by-Step Instructions:**
    1. Identify a **mandatory disclosure** (e.g., arbitration, consumer rights, financial terms).
    2. Move the disclosure **to a section where it would not normally be found**.
    3. Ensure that the relocation **reduces its enforceability or clarity**.
    4. Use **online legal databases or verified government resources** to **find a real law** that is contradicted by this relocation.
    5. For that contradiction, extract:
    - **The citation/title of the law** (e.g., "29 CFR § 516.2" or "CA Labor Code § 201")
    - **Two direct links (URLs)** to official or government sources for the law (**these links must be accessible. The relevant citation should be inside this URL. The page should be found - we should not get a Page Not Found or similar error.**)
    - **The first URL must come from an official government or legislature domain (ends in .gov, .mil, .state.<XX>.us, or one of uscode.house.gov, ecfr.gov, govinfo.gov, law.cornell.edu). Use the most recent version of the statute/regulation.**
    - **The second URL should be a secondary link to serve as a backup in case the first link falters, but we wish for that to not happen.**
    - **A brief explanation** of how the modified text contradicts the law (**law_explanation**)
    6. Make 2–3 such perturbations for the file.
    7. Output only the structured JSON object as shown below.
    
    ---
### **Examples of Misplaced Legal Disclosures:**

**Example 1:**
- **Original (Under 'Dispute Resolution'):** "In case of disputes, arbitration shall be required."
- **Modified (Moved to 'Optional Clauses'):** "In case of disputes, arbitration shall be required."
- **Explanation:** Some jurisdictions require **arbitration clauses to be in dispute resolution sections**.

**Example 2:**
- **Original (Under 'Employee Rights'):** "All employees are entitled to whistleblower protections."
- **Modified (Moved to 'Company Policies (Optional)'):** 
- **Explanation:** **Federal whistleblower laws** require **clear disclosure in a dedicated section**.

**Example 3:**
- **Original (Under 'Finance Terms'):** "Loan agreements must include an APR disclosure."
- **Modified (Moved to 'Supplementary Information'):** 
- **Explanation:** The **Truth in Lending Act (TILA)** mandates **explicit placement of APR disclosures**.

**Example 4:**
- **Original (Under 'Cancellation Policy'):** "Customers have the right to cancel subscriptions within 14 days."
- **Modified (Moved to 'Marketing and Promotions'):** 
- **Explanation:** Some consumer protection laws **require cancellation rights to be explicitly stated**.

**Example 5:**
- **Original (Under 'Privacy and Security'):** "Data privacy rights are outlined in the Privacy Policy."
- **Modified (Moved to 'Miscellaneous Terms'):** 
- **Explanation:** **GDPR and CCPA** require privacy rights to be **easily accessible**.

**Example 6:**
- **Original:** "In the event of a dispute, arbitration shall be the exclusive method of resolution, and no party shall have the right to seek litigation in any jurisdiction. The arbitration process shall follow the American Arbitration Association’s rules and be binding upon both parties."
- **Modified:** "Arbitration shall be the method of resolution in the event of a dispute, and no party shall have the right to seek litigation, but the method of resolution shall be determined by mutual agreement."
- **Explanation:** The restructuring of the sentence **introduces contradiction**—the first part states arbitration is **mandatory**, but the latter part allows for 'mutual agreement' on dispute resolution. This contradicts the original clause's **strict arbitration requirement**, leading to uncertainty in enforcement.

---
### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Structural Flaws - Legal Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "contradicted_law": "SPECIFIC LAW OR REGULATION BEING VIOLATED",
                "law_citation": "TITLE OR SECTION OF THE LAW (e.g., '29 CFR § 516.2')",
                "law_url1": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_1"
                ],
                "law_url2": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_2"
                ],
                "law_explanation": "HOW AND WHY THIS MODIFICATION CONTRADICTS THAT LAW",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}

Below is the original legal text:
-------------------
{original_text}
-------------------

Now, return **ONLY** the structured JSON object with the modified text and explanation.
"""


## Inconsistencies Legal

In [ ]:
f"""
You are an employment law specialist ensuring that contractual deadlines comply with legal regulations. Your task is to modify a timeline in the contract so that it contradicts state or federal laws.

Before modifying the text:
- **Read the file** to determine what city, state, or country the contract applies to.
- If the jurisdiction is unclear, default to **United States law**.
- Make sure that when taking the original texts, there should be no jumps between sentences. Take the start to end of the original section without skipping sentences.

### **Definition:**
Inconsistencies arise when **time-sensitive obligations** in a contract do not align with legal requirements. A **legal contradiction** in this category happens when a contract sets **a deadline or requirement that violates federal or state law**, making the contractual terms unenforceable or illegal.

### **Step-by-Step Instructions:**
    1. Identify a **contractual deadline** that is regulated by law (e.g., payment terms, claims deadlines, notice periods).
    2. Modify the deadline to **conflict with state or federal legal requirements**.
    3. Ensure that the change **creates non-compliance** with regulatory standards.
    4. Use **online legal databases or verified government resources** to **find a real law** that is contradicted by this new timeline.
   5. For that contradiction, extract:
   - **The citation/title of the law** (e.g., "29 CFR § 516.2" or "CA Labor Code § 201")
   - **Two direct links (URLs)** to official or government sources for the law (**these links must be accessible. The relevant citation should be inside this URL. The page should be found - we should not get a Page Not Found or similar error.**)
   - **The first URL must come from an official government or legislature domain (ends in .gov, .mil, .state.<XX>.us, or one of uscode.house.gov, ecfr.gov, govinfo.gov, law.cornell.edu). Use the most recent version of the statute/regulation.**
   - **The second URL should be a secondary link to serve as a backup in case the first link falters, but we wish for that to not happen.**
   - **A brief explanation** of how the modified text contradicts the law (**law_explanation**)
    6. Make 2–3 such perturbations for the file.
    7. Output only the structured JSON object as shown below.
    
    ---
### **Examples of Conflicts with Regulatory Timelines:**

**Example 1:**
- **Original:** "Employees must submit harassment claims within 15 days."
- **Modified:** "Employees must submit harassment claims within 5 days."
- **Explanation:** Some states (e.g., California) require **at least 30 days** for harassment claims.

**Example 2:**
- **Original:** "Landlords must return security deposits within 21 days of lease termination."
- **Modified:** "Landlords must return security deposits within 60 days of lease termination."
- **Explanation:** Many states **mandate 14-30 days** for deposit refunds.

**Example 3:**
- **Original:** "Customers have the right to cancel a contract within 10 days of signing."
- **Modified:** "Customers may cancel contracts within 48 hours."
- **Explanation:** Some **consumer protection laws** require **at least 7-10 days** for cancellations.

**Example 4:**
- **Original:** "Workers must receive final wages within 72 hours of termination."
- **Modified:** "Workers will receive final wages at the company's discretion."
- **Explanation:** Federal and state laws **require clear final paycheck deadlines**.

**Example 5:**
- **Original:** "Loan repayment plans must allow at least 90 days for late payments before default."
- **Modified:** "Loan repayment plans may declare default after 30 days of non-payment."
- **Explanation:** Some **loan regulations** require longer grace periods.

**Example 6:**
- **Original:** "The term 'Confidential Information' shall refer to any proprietary business, financial, and technical data... Confidential Information shall be protected for a period of five (5) years from the date of disclosure."
- **Modified:** "The term 'Confidential Information' shall refer to sensitive business information disclosed by one party to the other. Each party shall determine what constitutes Confidential Information based on its internal policies. Confidentiality obligations shall remain in effect for a commercially reasonable period."
- **Explanation:** Replacing a **clear five-year obligation** with "commercially reasonable period" creates **uncertainty**. Allowing each party to define confidentiality based on internal policies **contradicts** standard contract law principles requiring **definitive** confidentiality terms.

---
### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Inconsistencies - Legal Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "contradicted_law": "SPECIFIC LAW OR REGULATION BEING VIOLATED",
                "law_citation": "TITLE OR SECTION OF THE LAW (e.g., '29 CFR § 516.2')",
                "law_url1": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_1"
                ],
                "law_url2": [
                    "OFFICIAL_LEGAL_REFERENCE_URL_2"
                ],
                "law_explanation": "HOW AND WHY THIS MODIFICATION CONTRADICTS THAT LAW",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}

Below is the original legal text:
-------------------
{original_text}
-------------------

Now, return **ONLY** the structured JSON object with the modified text and explanation.
"""
